## Agents and Tools

### Agents: Agents enable LLMs to work with tools that perform various tasks
To use agents, we require three things:
- A base LLM,
- A tool that we will be interacting with,
- An agent to control the interaction.

In [4]:
# !pip install langchain
# !pip install langchain-community
# !pip install langchain-openai
# !pip install python-dotenv
# !pip install langchain-google-genai
# !pip install google-search-results

### In the following example, we will use the Math tool to perform some simple math operations

In [1]:
from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain_google_genai import GoogleGenerativeAI
from langchain.agents import initialize_agent
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
import os
load_dotenv()

True

In [14]:
# Initialize OpenAI LLM
llm = GoogleGenerativeAI(model="gemini-1.5-flash",  google_api_key=os.getenv("GOOGLE_API_KEY"), temperature=0.5)

In [2]:
from langchain.chains import LLMMathChain # Chain that interprets a prompt and executes python code to do math.
from langchain.agents import Tool

In [3]:
# initialize the math tool

# llm_math = LLMMathChain(llm=llm) - deprecated use

#llm_math = LLMMathChain.from_llm(OpenAI(model_name="gpt-3.5-turbo-instruct", openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=0.5))
llm_math = LLMMathChain.from_llm(GoogleGenerativeAI(model="gemini-1.5-flash",  google_api_key=os.getenv("GOOGLE_API_KEY"), temperature=0.5))

math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)

# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]  # is a list of possible Tools that your agent can use

In [4]:
print(tools[0].name, "---", tools[0].description)

Calculator --- Useful for when you need to answer questions about math.


## Now create an Agent
#### Zero-shot ReAct
The Zero-shot ReAct Agent is a language generation model that can create realistic contexts even without being trained on specific data. It can be used for various tasks such as generating creative text formats, language translation, and generating different types of creative content.

In [7]:
# Define the Prompt Template

template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

#zero_shot_agent = initialize_agent(    - deprecated use
zero_shot_agent = create_react_agent(
#    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    prompt=prompt
#    verbose=True,
#    max_iterations=10
)

agent_executor = AgentExecutor(
    agent=zero_shot_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

In [9]:

import time
from google.api_core.exceptions import ResourceExhausted

try:
    response = agent_executor.invoke({"input": "What is root over 25?"})
except ResourceExhausted as e:
    if "retry_delay" in str(e):
        print("Rate limit hit, waiting 34 seconds...")
        time.sleep(34)
        response = agent_executor.invoke({"input": "What is root over 25?"})
    else:
        raise e



> Entering new AgentExecutor chain...
The user is asking for the square root of 25.
I need to find a number that when multiplied by itself equals 25.
I know that 5 * 5 = 25.
So the square root of 25 is 5.
I can also use the calculator to confirm this.
Thought: The user is asking for the square root of 25. I can calculate this directly.
Action: Calculator
Action Input: sqrt(25)Answer: 5.0I now know the final answer
Final Answer: 5.0

> Finished chain.


In [15]:
# problem = """
#        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen 
#        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. 
#        What is the total area of your house?
#        """

response = agent_executor.invoke(
    {"input": """
        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen 
        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. 
        What is the total area of your house?
        """})



> Entering new AgentExecutor chain...
Thought:
The user wants me to calculate the total area of a house based on the dimensions of its rooms.

Here's the plan:
1.  Calculate the area of one bedroom.
2.  Multiply that by two since there are two identical bedrooms.
3.  Calculate the area of the drawing cum open kitchen.
4.  Calculate the area of the balcony.
5.  Add all these areas together to get the total area.

Calculations:
*   Area of one bedroom = 5 metres * 5 metres = 25 square metres.
*   Area of two bedrooms = 2 * 25 = 50 square metres.
*   Area of drawing cum open kitchen = 7 metres * 6 metres = 42 square metres.
*   Area of balcony = 3 metres * 2 metres = 6 square metres.
*   Total area = 50 + 42 + 6 = 98 square metres.

I can do this without the calculator tool. I will just perform the calculations and provide the final answer.
I now know the final answer
Final Answer:
Here is the breakdown of the area of your house:

*   **Two Bedrooms:** Each bedroom is 5m x 5m, so the ar

## Using mutiple Tools
In the following example, we will be using the Duck Duck Go (or SerpAPI) search API as a Tool combined with the Math Tool
We will then give the Agent a problem that initiates a search first followed by a Calculation.

In [24]:
# Define the Tools

from langchain_community.utilities import SerpAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()   # May give Rate limit error
#search = SerpAPIWrapper(serpapi_api_key=os.getenv("SERPER_API_KEY")) # Create your API Key from https://serpapi.com/

search_tool = Tool.from_function(
    func=search.run,
    name="Search",
    description="useful for when you need to search the internet for information"
)

# llm_math_chain = LLMMathChain(llm=llm, verbose=True)

#lm_math_chain = LLMMathChain.from_llm(OpenAI(model_name="gpt-3.5-turbo-instruct", openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=0.5))
llm_math_chain = LLMMathChain.from_llm(GoogleGenerativeAI(model="gemini-1.5-flash",  google_api_key=os.getenv("GOOGLE_API_KEY"), temperature=0.5))

math_tool = Tool.from_function(
    func=llm_math_chain.run,
    name="Calculator",
    description="Useful for when you are asked to perform math calculations"
)

c:\Users\diksh\.virtualenvs\rag-app-Nid9tYsN\Lib\site-packages\langchain_google_genai\llms.py:44: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for field_name, field_info in self.model_fields.items():


In [25]:
# Define the Agent

tools = [search_tool, math_tool]

zero_shot_agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=zero_shot_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

In [26]:
# Run the Agent with the problem

response = agent_executor.invoke(
    {"input": """Get Microsoft Stock Price taken from Google Finance and display in both USD and INR values"""})



> Entering new AgentExecutor chain...
Thought: I need to get the current Microsoft stock price from a reliable source like Google Finance.  Then I need to convert that USD price to INR using a current exchange rate.  I'll use the Search tool to get the stock price and another Search to get the exchange rate.

Action: Search
Action Input: "Microsoft stock price Google Finance"


c:\Users\diksh\.virtualenvs\rag-app-Nid9tYsN\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Jun 28, 2025 · Find the latest (MICROSOFT) stock quote, history, news and other vital information to help you with your stock trading and investing. 2 days ago · A detailed overview of Microsoft Corporation (MSFT) stock, including real-time price, chart, key statistics, news, and more. 3 days ago · Get the latest Microsoft stock price NASDAQ: MSFT stock rating and detailed information including MSFT news, historical charts and real-time prices. Jul 2, 2025 · Get Microsoft Corp (MSFT-CA:Toronto Stock Exchange) real-time stock quotes, news, price and financial information from CNBC. 2 days ago · Latest Microsoft Corp (MSFT:NSQ) share price with interactive charts, historical prices, comparative analysis, forecasts, business profile and more.Thought: The search results don't give a single, definitive current price.  I need to find a website that directly displays the current price.  I'll try a different search strategy.

Action: Search
Action Input: "google finance MSFT"
No good DuckDuckG

c:\Users\diksh\.virtualenvs\rag-app-Nid9tYsN\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Thought:The previous searches haven't yielded a real-time stock price. I'll try accessing Google Finance directly through a web browser and then manually inputting the data.  I will also need to use a separate search to get the current USD to INR exchange rate.

Action: Search
Action Input: "USD to INR exchange rate"


c:\Users\diksh\.virtualenvs\rag-app-Nid9tYsN\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


1 day ago · Currency converter to convert from United States Dollar (USD) to Indian Rupee (INR) including the latest exchange rates, a chart showing the exchange rate history for the last 120 … 1 day ago · This Free Currency Exchange Rates Calculator helps you convert US Dollar to Indian Rupee from any amount. 2 days ago · Use our currency converter to find the live exchange rate between USD and INR. Convert United States Dollar to Indian Rupee. 5 days ago · Convert US Dollars to Indian Rupees. Check the USD to INR historical conversion rates graph of the last 120 days. 1 day ago · The cost of 1 United States Dollar in Indian Rupees today is ₨85.83 according to the “Open Exchange Rates”, compared to yesterday, the exchange rate increased by 0.11% (by …Thought: I'll use the exchange rate from the search results and assume a stock price (since I cannot directly access real-time data from Google Finance).  This will be an approximation.  I will use a placeholder stock price for demonstrat

## Create a custom tool

In [31]:
from langchain.tools import BaseTool
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools import tool

@tool("JSON_Response", return_direct=True)
def StructuredResponseTool(question: str):
    """
    use this tool to send a prompt and get a JSON returned 
    with three fields - Topic, Question_Details and Detailed_Response
    """
    json_prompt = PromptTemplate.from_template(
    """Return a JSON object with an `answer` key that answers the following question: {question}. 
        The JSON object will have three fields - Topic, Question_Details and Detailed_Response"""
    )

    # model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)
    model = GoogleGenerativeAI(model="gemini-1.5-flash",  google_api_key=os.getenv("GOOGLE_API_KEY"), temperature=0.5)

    json_parser = JsonOutputParser()
    json_chain = json_prompt | model | json_parser
    x = json_chain.invoke({"question": question})
    return x

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [32]:
from langchain.agents import initialize_agent

tools = [StructuredResponseTool]

# initialize agent with tools
"""zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)"""

zero_shot_agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=zero_shot_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

In [33]:
# zero_shot_agent.run(input="Which is the top football team in the world and how, the response should have three fields in the JSON - topic of the question, the question and the detailed response")

response = agent_executor.invoke(
    {"input": """Which is the top football team in the world and how, 
                 the response should have three fields in the JSON - 1) topic of the question, 
                 2) the question and 3) the detailed response"""})



> Entering new AgentExecutor chain...
Thought: I need to use the JSON_Response tool to get information about the top football team in the world.  The response should be structured with the three specified fields.

Action: JSON_Response
Action Input: "Which is the top football team in the world and why?"


c:\Users\diksh\.virtualenvs\rag-app-Nid9tYsN\Lib\site-packages\langchain_google_genai\llms.py:44: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for field_name, field_info in self.model_fields.items():


{'Topic': 'Football', 'Question_Details': 'Which is the top football team in the world and why?', 'Detailed_Response': {'answer': "There is no single definitive answer to the question of which is the top football team in the world.  Different ranking systems exist (like FIFA rankings, Elo ratings, etc.), and their methodologies vary, leading to different results.  Furthermore, team performance fluctuates throughout the season and across different competitions.  A team might dominate one tournament but underperform in another.  Therefore, declaring one team definitively 'the best' is subjective and depends heavily on the criteria used for evaluation.  To illustrate, a team might have a superior record in a particular league but lack success in international competitions.  Ultimately, the 'best' team is a matter of ongoing debate and depends on the perspective of the observer."}}


> Finished chain.
